In [6]:
import mne
import numpy as np
import pandas as pd
import matplotlib
import seaborn as sns


In [1]:
import mne
import numpy as np
import pandas as pd
import matplotlib
import seaborn as sns

%matplotlib qt

def read_file(path):
    r = mne.io.read_raw_bdf(path, preload=False)
    
    # Define signal info
    n_time_samps = r.n_times
    time_secs = r.times
    ch_names = r.ch_names
    n_chan = len(ch_names)  # note: there is no raw.n_channels attribute
    
    
    channels_to_keep = ["A1", "A2", "A3", "A4", "A5", "A6", "A7", "A8", "A9", "A10",
            "A11", "A12", "A13", "A14", "A15", "A16", "A17", "A18", "A19", "A20",
            "A21", "A22", "A23", "A24", "A25", "A26", "A27", "A28", "A29", "A30",
            "A31", "A32", "B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8", "B9", "B10",
            "B11", "B12", "B13", "B14", "B15", "B16", "B17", "B18", "B19", "B20",
            "B21", "B22", "B23", "B24", "B25", "B26", "B27", "B28", "B29", "B30", "B31", "B32"]

    channels_to_exclude_str = [ch for ch in r.ch_names if ch not in channels_to_keep + ['Status']]
    
    
    raw = mne.io.read_raw_bdf(path, exclude = channels_to_exclude_str ,  preload=True)
    
    list_chan_name = { 'A3': 'AF3',
     'A7': 'F7', 'A6': 'F5','A5': 'F3', 'A4': 'F1',
     'A8': 'FT7', 'A9': 'FC5','A10': 'FC3', 'A11': 'FC1',
     'A15': 'T7', 'A14': 'C5','A13': 'C3', 'A12': 'C1',
     'A16': 'TP7', 'A17': 'CP5','A18': 'CP3', 'A19': 'CP1', 'A32': 'CPz',
     'A24': 'P9', 'A23': 'P7','A22': 'P5', 'A21': 'P3', 'A20': 'P1','A31': 'Pz',
     'A25': 'PO7', 'A26': 'PO3','A30': 'POz', 
     'A27': 'O1',   'A29': 'Oz', 
     'A28': 'Iz', 
     'B1': 'Fpz','B2': 'Fp2',
     'B5': 'AFz', 'B4': 'AF4','B3': 'AF8',
     'B6': 'Fz', 'B7': 'F2','B8': 'F4', 'B9': 'F6','B10': 'F8',
     'B15': 'FCz', 'B14': 'FC2','B13': 'FC4', 'B12': 'FC6','B11': 'FT8',
     'B16': 'Cz', 'B17': 'C2','B18': 'C4', 'B19': 'C6','B20': 'T8',
     'B24': 'CP2', 'B23': 'CP4','B22': 'CP6', 'B21': 'TP8',
     'B25': 'P2', 'B26': 'P4','B27': 'P6', 'B28': 'P8','B29': 'P10',
     'B31': 'PO4', 'B30': 'PO8',
     'B32': 'O2'
     }

    _ = raw.rename_channels(list_chan_name)
    print(raw.info)
    
    return raw

    
def highlow_pass(raw):
    # We cut off at one Hertz, because we will be looking at the alpha and beta frequency bands that start
# from the 8-10 Hz. Apply high-pass filter
    raw.filter(l_freq=1, h_freq=None, fir_design='firwin', filter_length='auto',
           l_trans_bandwidth='auto', h_trans_bandwidth='auto', method='fir',
           phase='zero', fir_window='hamming', verbose=True)

# Apply low-pass filter if needed
    raw.filter(l_freq=None, h_freq=40, fir_design='firwin', filter_length='auto',
           l_trans_bandwidth='auto', h_trans_bandwidth='auto', method='fir',
           phase='zero', fir_window='hamming', verbose=True)

    return raw 
    
    
def plotsignal(signal, channels, pick_min, pick_max):
    
    ch_names = signal.ch_names
    
    
    picks = ch_names[pick_min:pick_max]  # Pick the first 20 EEG channels
    signal.plot(n_channels=channels, picks=picks, events = False, title='EEG Data for the Channels')
    
    
    
def rename_events(raw):
    # Find events and drop unnecessary   
    events_ID = mne.find_events(raw, stim_channel="Status")
    events = mne.pick_events(events_ID, exclude = [3,5,40,42,62])

    # Define epoch parameters
    tmin = -2  
    tmax = 3   

    # Epoch  signal
    epochs = mne.Epochs(raw, events, tmin=tmin, tmax=tmax, baseline = None)
    
    unsuccessful_event_index = None # For sanity check


    for i, event in enumerate(epochs.events[:-1]):  # Iterate over all events except the last one
        if event[2] == 10:
            next_event = epochs.events[i+1]
            while next_event[2] == 61:
                next_event[2] = 610
                i += 1  # Increment i to move to the next event
                if i < len(epochs.events) - 1:
                    next_event = epochs.events[i+1]
                else:
                    break  # Break the loop if we reached the end of events
                
                
        elif event[2] == 20:
            next_event = epochs.events[i+1]
            while next_event[2] == 61:
                next_event[2] = 620
                i += 1
                if i < len(epochs.events) - 1:
                    next_event = epochs.events[i+1]
                else:
                    break
                
                
        elif event[2] == 30:
            next_event = epochs.events[i+1]
            while next_event[2] == 61:
                next_event[2] = 630
                i += 1
                if i < len(epochs.events) - 1:
                    next_event = epochs.events[i+1]
                else:
                    break
        else:
            unsuccessful_event_index = i

    # Check if any events with trigger value 61 are still present
    if any(event[2] == 61 for event in epochs.events):
        print("Renaming was unsuccessful :(")
        if unsuccessful_event_index is not None:
            print("Problematic event index:", unsuccessful_event_index)
            print("Problematic event details:", epochs.events[unsuccessful_event_index])
            
    #raw.plot(
    #events=events,
    #start=5,
    #duration=10,
    #color="gray",
    #event_color={41: "r", 10: "g", 20: "b", 30: "m"}, title = "Raw filtered signal with 4 conditions highlighted")
    
    #  Might introduce edge artifacts for each epoch.
    epochs.load_data()
    epochs_rs = epochs.resample(sfreq=256)

    # Print events 
    # # Assuming you have your epochs object named epochs
    # Get the event IDs present in your epochs
    event_ids = epochs_rs.events[:, -1]

    # Print unique event IDs
    unique_event_ids = set(event_ids)
    print("Unique Event IDs:", unique_event_ids)

    # Print counts of each event ID
    for event_id in unique_event_ids:
        count = (event_ids == event_id).sum()
        print("Event ID:", event_id, "Count:", count)
        
    return epochs
    


sample_fname = "C:/Users/Mabel Ife/OneDrive - Danmarks Tekniske Universitet/Dokumenter/Master in AI/Final Thesis/Data/subj_07_070223_task.bdf"
raw_filt = highlow_pass(read_file(sample_fname))    

# Epoch and events   



    
    
    
    
    
    
    
    

    
        
    

# some examples of raw.info

Extracting EDF parameters from C:\Users\Mabel Ife\OneDrive - Danmarks Tekniske Universitet\Dokumenter\Master in AI\Final Thesis\Data\subj_07_070223_task.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\Mabel Ife\OneDrive - Danmarks Tekniske Universitet\Dokumenter\Master in AI\Final Thesis\Data\subj_07_070223_task.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 4517887  =      0.000 ...  2206.000 secs...
<Info | 8 non-empty values
 bads: []
 ch_names: A1, A2, AF3, F1, F3, F5, F7, FT7, FC5, FC3, FC1, C1, C3, C5, T7, ...
 chs: 64 EEG, 1 Stimulus
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 417.0 Hz
 meas_date: 2023-02-07 14:46:06 UTC
 nchan: 65
 projs: []
 sfreq: 2048.0 Hz
 subject_info: 1 item (dict)
>
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pas

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:   15.2s finished


Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 677 samples (0.331 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:   13.2s finished


In [2]:
# create epochs, renames events, resample to 256HZ
epochs_rs = rename_events(raw_filt)

Trigger channel Status has a non-zero initial value of {initial_value} (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
4237 events found on stim channel Status
Event IDs: [ 3  5 10 20 30 40 41 42 61 62]
Not setting metadata
1707 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1707 events and 10241 original time points ...
0 bad epochs dropped
Unique Event IDs: {610, 41, 10, 620, 20, 630, 30}
Event ID: 610 Count: 450
Event ID: 41 Count: 540
Event ID: 10 Count: 30
Event ID: 620 Count: 301
Event ID: 20 Count: 30
Event ID: 630 Count: 326
Event ID: 30 Count: 30


In [3]:
# Copy epochs to drop_channel as it is no reversible 
ep_ds = epochs_rs.copy()

# Highlight bad channels # we keep the flat one as it will be fixed after averaging 
_ = ep_ds.info["bads"].extend(['TP8'])  

# Set montage and interpolate bads 
ep_ds.set_montage('standard_1020')
ep_interp_mne = ep_ds.interpolate_bads(
    reset_bads=False, method='MNE', verbose=True
)


#ep_interp_mne.plot(picks=("grad", "eeg"))


Setting channel interpolation method to {'eeg': 'MNE'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 95.6 mm
    Computing dot products for 63 EEG channels...
    Computing cross products for 63 → 64 EEG channels...
    Preparing the mapping matrix...
    Truncating at 63/63 components and regularizing with α=1.0e-01


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


In [19]:
# use the average of all channels as reference
#ep_avg_ref = ep_interp_mne.copy().set_eeg_reference(ref_channels="average")


# Participant ID code
p_id = 'sub-007'
data_dir = 'C:/Users/Mabel Ife/OneDrive - Danmarks Tekniske Universitet/Dokumenter/Master in AI/Final Thesis/Data/epochs/'

#ica_motor_comp.save(data_dir + p_id + '-ica.fif', 
#        overwrite=True)


ep_avg_ref.save(data_dir + p_id + '-epo.fif', overwrite=True)


Overwriting existing file.


Overwriting existing file.


### Getting muscle components from ICA  



In [6]:
# Automated algorithm to find muscle components
# Run ICA

ica = mne.preprocessing.ICA(
    n_components=30, method="fastica", max_iter="auto", random_state=97)
ica.fit(ep_avg_ref)




Fitting ICA to data using 63 channels (please be patient, this may take a while)
Selecting by number: 30 components
Fitting ICA took 259.1s.


Method,fastica
Fit parameters,algorithm=parallelfun=logcoshfun_args=Nonemax_iter=1000
Fit,56 iterations on epochs (2184960 samples)
ICA components,30
Available PCA components,63
Channel types,eeg
ICA components marked for exclusion,—


In [7]:
#%matplotlib qt

# Visualise ICA components
montage = mne.channels.make_standard_montage("standard_1005")
ep_avg_ref.set_montage(montage)

ica.plot_sources(ep_avg_ref) # plot the raw signal 

#scalp field topographies 
ica.plot_components() # plot a Topomap 
ica.plot_properties(ep_avg_ref, picks=[8,9], log_scale=False)
# plot specific components 


Not setting metadata
1707 matching events found
No baseline correction applied
0 projection items activated
Using matplotlib as 2D backend.
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
1707 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1707 matching events found
No baseline correction applied
0 projection items activated


[<Figure size 700x600 with 6 Axes>, <Figure size 700x600 with 6 Axes>]

For the subject 7, we found the two components 8 and 9. 

In [8]:
# Participant ID code
p_id = 'sub-007'
data_dir = 'C:/Users/Mabel Ife/OneDrive - Danmarks Tekniske Universitet/Dokumenter/Master in AI/Final Thesis/Data/Ica_files/'

#ica_motor_comp.save(data_dir + p_id + '-ica.fif', 
#        overwrite=True)

ica.save(data_dir + p_id + '-ica.fif', 
        overwrite=True)

Writing ICA solution to C:\Users\Mabel Ife\OneDrive - Danmarks Tekniske Universitet\Dokumenter\Master in AI\Final Thesis\Data\Ica_files\sub-007-ica.fif...


Method,fastica
Fit parameters,algorithm=parallelfun=logcoshfun_args=Nonemax_iter=1000
Fit,56 iterations on epochs (2184960 samples)
ICA components,30
Available PCA components,63
Channel types,eeg
ICA components marked for exclusion,—


## ICA and Wavelet analysis 

In [27]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.colors import TwoSlopeNorm

import mne
from mne.datasets import eegbci
from mne.io import concatenate_raws, read_raw_edf
from mne.stats import permutation_cluster_1samp_test as pcluster_test
from mne.time_frequency import tfr_multitaper, tfr_morlet

In [ ]:
# Restore ICA solution from fif file.

p_id = 'sub-007' # Participant ID code
data_dir = 'C:/Users/Mabel Ife/OneDrive - Danmarks Tekniske Universitet/Dokumenter/Master in AI/Final Thesis/Data/'
components_from_ICA = mne.preprocessing.read_ica(data_dir + p_id + '-ica.fif', verbose=None)


Reading C:/Users/Mabel Ife/OneDrive - Danmarks Tekniske Universitet/Dokumenter/Master in AI/Final Thesis/Data/sub-007-ica.fif ...
Now restoring ICA solution ...
Ready.


In [21]:
#apply the ICA decomposition (excluding the marked ICs) to the epochs
epochs_postica = ica.apply(ep_avg_ref.copy())

Applying ICA to Epochs instance
    Transforming to ICA space (30 components)
    Zeroing out 0 ICA components
    Projecting back using 63 PCA components


In [24]:
# Split epochs data into four conditions based on events


condition_1_epochs = epochs_postica['41']
#condition_2_epochs = epochs_postica['10']
#condition_3_epochs = epochs_postica['20']
#condition_4_epochs = epochs_postica['30']

# Extract independent components
components_1 = ica.get_sources(ep_avg_ref['41'])
components_2 = ica.get_sources(ep_avg_ref['10'])
components_3 = ica.get_sources(ep_avg_ref['20'])
components_4 = ica.get_sources(ep_avg_ref['30'])



In [20]:
# waveanalysis

#del condition_1_epochs,condition_2_epochs,condition_3_epochs,condition_4_epochs, components_1,components_2,components_3, ica,ica_motor_comp
#del ICA


from mne.time_frequency import morlet, fwhm

# Define parameters
freqs = np.arange(2, 31) 
#freqs = 10 # Frequencies of interest from 1 to 30 Hz
n_cycles = 2 + 0.125 * freqs # Number of cycles 



# Define time window
tmin = -1  # Start time
tmax = 2   # End time
sfreq = 256 # Sampling frequency

#.morlet(freqs=freqs, n_cycles=n_cycles, sfreq=sfreq, zero_mean=False)


mne.time_frequency.tfr_morlet(ica, freqs, n_cycles, use_fft=False, return_itc=False, decim=1, n_jobs=None, zero_mean=True, average=False, output='power', verbose=None)


# Perform time-frequency analysis with Morlet wavelet
#power = morlet(freqs=freqs, n_cycles=n_cycles, sfreq=sfreq, zero_mean=False, sigma=None)

# Plot time-frequency representation
#power_1.plot_joint(timefreqs=(tmin, tmax))

TypeError: inst must be Epochs or Evoked

In [28]:
#del ep_avg_ref, epochs_rs
p = tfr_morlet(condition_1_epochs, freqs, n_cycles, use_fft=False, return_itc=False, decim=1, n_jobs=None, zero_mean=True, average=False, output='power', verbose=None)

MemoryError: Unable to allocate 9.41 GiB for an array with shape (63, 540, 29, 1280) and data type float64

In [11]:
# Perform time-frequency analysis with Morlet wavelet
power2 = morlet(ica, freqs=freqs, n_cycles=n_cycles, sfreq=sfreq, zero_mean=False, sigma=None)

TypeError: morlet() got multiple values for argument 'sfreq'

In [17]:
del ep_ds,ep_interp_mne,epochs_postica,